In [2]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)

sample_size = 5000

In [3]:
## take creation_date, tags from bigquery-public-data.stackoverflow.posts_questions
## only between 1/1/2019 and 31/12/2019
## misura il conteggio di ogni tag e mostra i 10 più famosi

query = '''
    SELECT RAND() as r, creation_date, tags FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE creation_date<TIMESTAMP("2019-12-31 23:59:00+00")
    AND creation_date>TIMESTAMP("2017-01-01 00:00:00+00")
    AND (REGEXP_CONTAINS(tags, r"vue.js") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular"))
    ORDER BY r
    LIMIT ''' + str(sample_size)

df = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)



In [4]:
myDf = df

# i[0] = 'tag|tag2'

def setTag(x):
    if(x.find('react') != -1):
        return 'react'
    elif(x.find('angular') != -1):
        return 'angular'
    elif(x.find('vue.js') != -1):
        return 'vue.js'

def setDate(x):
    return x.year;

myDf['tags'] = myDf['tags'].map(setTag)
myDf['creation_date'] = myDf['creation_date'].map(setDate)

In [5]:
myDf.sort_values(by='creation_date')

,r,creation_date,tags
2499,0.005224,2017,angular
1387,0.002936,2017,react
1385,0.002933,2017,angular
4075,0.008561,2017,angular
2782,0.005861,2017,angular
...,...,...,...
2549,0.005327,2019,react
2554,0.005342,2019,react
2555,0.005345,2019,react
2542,0.005309,2019,react


In [17]:
tagDf = myDf.drop(columns='r')
tagDf['count'] = 1
tagDf = tagDf.groupby(['tags', 'creation_date']).count().unstack()

print(tagDf['count'])
hist = px.histogram(tagDf, x=['view.js', 'angular', 'react'], color=[2017,2018,2019], y='count', title="JS Framework Interests in 2019 (based on 5k sample)", barmode="group")
hist.show()

creation_date  2017  2018  2019
tags                           
angular        1030   809   652
react           474   686   897
vue.js           86   168   198


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().